In [ ]:
import torch
from neuralforecast import NeuralForecast

nf = NeuralForecast.load(path='../models/neuralforecast_models')

pred = nf.predict().reset_index()

In [ ]:
from sqlalchemy import create_engine
from neuralforecast.losses.numpy import mae, rmse, mape
import pandas as pd

# 创建数据库引擎
table_hk_daily = "hk_customs_daily"
database_url = "sqlite:///../data/data.sqlite"
engine = create_engine(database_url)

cross_val = pd.read_sql('model_evaluation', con=engine)
cross_val.head()
horizon = 30

model_names = ['AutoLSTM', 'AutoInformer', 'AutoAutoformer', 'AutoPatchTST', 'AutoiTransformer']


def eval_accuracy(df):
    unique_ids = df['unique_id'].unique()
    for unique_id in unique_ids:
        df_tmp = df[df['unique_id'] == unique_id]
        print(unique_id)
        y_true = df_tmp['y']
        for model_name in model_names:
            y_pred = df_tmp[model_name]
            print(
                f'{model_name} -- mae: {mae(y_true, y_pred)},rmse: {rmse(y_true, y_pred)}, mape: {mape(y_true, y_pred)}')


eval_accuracy(cross_val)